# VDBFusion example

This notebook is rather a small example to get you started with the use of VDBFusion.

If you use our work please consider citing us:

```bibtex
@article{vizzo2022sensors,
  author         = {Vizzo, Ignacio and Guadagnino, Tiziano and Behley, Jens and Stachniss, Cyrill},
  title          = {VDBFusion: Flexible and Efficient TSDF Integration of Range Sensor Data},
  journal        = {Sensors},
  volume         = {22},
  year           = {2022},
  number         = {3},
  article-number = {1296},
  url            = {https://www.mdpi.com/1424-8220/22/3/1296},
  issn           = {1424-8220},
  doi            = {10.3390/s22031296}
}
```


## Step 1. Install vdbfusion

The first we need to check is that if we can install the `vdbfusion` library. If you ran into any issue at this step you should check our [github repository](https://github.com/PRBonn/vdbfusion) and submit an issue there.

In [ ]:
%pip install vdbfusion
import vdbfusion
print(f"VDBFusion version {vdbfusion.__version__} properly installed at {vdbfusion.__file__}")

## Step 2. Install example dependencies

While `vdbfusion` only depends on `numpy` the tools we will be using in this tutorial does not, therefore we will install some common dependencies and make sure we can import them

In [ ]:
%pip install trimesh pandas tqdm | grep -v 'already satisfied'

In [ ]:
from tqdm import tqdm
import pandas
import trimesh

## Step 3. Get some toy data

In order to get you started with the use of the library we will use a subset of the sequence `00` from the [KITTI Odometry dataset](http://www.cvlibs.net/datasets/kitti/eval_odometry.php). We selected the first 100 scans just to avoid downloading big datasets. If you plan to use the library for real work, you should rather consider the complete examples at the [examples repository](https://github.com/PRBonn/vdbfusion/examples)

In [ ]:
print("Downloading a small subset of the kitti odometry dataset")
!wget -c https://www.ipb.uni-bonn.de/html/software/vdbfusion/kitti-odometry.tar.gz -O - | tar -xz

## Step 4. Create Dataloader

In [ ]:
import os
import glob

import numpy as np
import pandas as pd

from trimesh import transform_points


class Config:
    # Data specific params
    apply_pose = True
    min_range = 2.0
    max_range = 70.0


class KITTIOdometryDataset:
    def __init__(self, kitti_root_dir: str, sequence: int):
        """Simple KITTI DataLoader to provide a ready-to-run example.

        Heavily inspired in PyLidar SLAM
        """
        # Config stuff
        self.sequence = str(int(sequence)).zfill(2)
        self.config = Config()
        self.kitti_sequence_dir = os.path.join(kitti_root_dir, "sequences", self.sequence)
        self.velodyne_dir = os.path.join(self.kitti_sequence_dir, "velodyne/")

        # Read stuff
        self.calibration = self.read_calib_file(os.path.join(self.kitti_sequence_dir, "calib.txt"))
        self.poses = self.load_poses(os.path.join(kitti_root_dir, f"poses/{self.sequence}.txt"))
        self.scan_files = sorted(glob.glob(self.velodyne_dir + "*.bin"))

    def __getitem__(self, idx):
        return self.scans(idx), self.poses[idx]

    def __len__(self):
        return len(self.scan_files)

    def scans(self, idx):
        return self.read_point_cloud(idx, self.scan_files[idx], self.config)

    def read_point_cloud(self, idx: int, scan_file: str, config: Config):
        points = np.fromfile(scan_file, dtype=np.float32).reshape((-1, 4))[:, :-1]
        points = points[np.linalg.norm(points, axis=1) <= config.max_range]
        points = points[np.linalg.norm(points, axis=1) >= config.min_range]
        points = transform_points(points, self.poses[idx]) if config.apply_pose else points
        return points

    def load_poses(self, poses_file):
        def _lidar_pose_gt(poses_gt):
            _tr = self.calibration["Tr"].reshape(3, 4)
            tr = np.eye(4, dtype=np.float64)
            tr[:3, :4] = _tr
            left = np.einsum("...ij,...jk->...ik", np.linalg.inv(tr), poses_gt)
            right = np.einsum("...ij,...jk->...ik", left, tr)
            return right

        poses = pd.read_csv(poses_file, sep=" ", header=None).values
        n = poses.shape[0]
        poses = np.concatenate(
            (poses, np.zeros((n, 3), dtype=np.float32), np.ones((n, 1), dtype=np.float32)), axis=1
        )
        poses = poses.reshape((n, 4, 4))  # [N, 4, 4]
        return _lidar_pose_gt(poses)

    @staticmethod
    def read_calib_file(file_path: str) -> dict:
        calib_dict = {}
        with open(file_path, "r") as calib_file:
            for line in calib_file.readlines():
                tokens = line.split(" ")
                if tokens[0] == "calib_time:":
                    continue
                # Only read with float data
                if len(tokens) > 0:
                    values = [float(token) for token in tokens[1:]]
                    values = np.array(values, dtype=np.float32)
                    # The format in KITTI's file is <key>: <f1> <f2> <f3> ...\n -> Remove the ':'
                    key = tokens[0][:-1]
                    calib_dict[key] = values
        return calib_dict

## Step 5. Run the fusion pipeline

In [ ]:
from vdbfusion import VDBVolume

# Create a VDB Volume to integrate scans
vdb_volume = VDBVolume(voxel_size=0.1, sdf_trunc=0.3, space_carving=False)

# You need to define your own Dataset.
dataset = KITTIOdometryDataset(kitti_root_dir="./kitti-odometry/dataset/", sequence=0)

for scan, pose in tqdm(dataset):
    vdb_volume.integrate(scan, pose)

## Step 6. Visualize the results

In order to store the results on this notebook, we use the trimesh library. But if you are running this locally, you can use `Open3D` or any other 3D library you like.

In [ ]:
# Extract a mesh from vdbfusion
vertices, triangles = vdb_volume.extract_triangle_mesh(fill_holes=True, min_weight=5.0)

mesh = trimesh.Trimesh(vertices=vertices, faces=triangles)
mesh.show()

## Where to go next?

If you think the library could be useful for your work, now is time to check our open source code at our [github](https://github.com/PRBonn/vdbfusion)